In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris

# 乱数のシードを設定
torch.manual_seed(0)

# irisデータセットの読み込み
iris = load_iris()

# 入力値と目標値を抽出
x = iris.data
t = iris.target

# Tensorに変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# 入力値と目標値をまとめる
# （[[[入力値1, 入力値2, ...], 目標値1], [[入力値1, 入力値2, ...], 目標値2], ...]）
dataset = torch.utils.data.TensorDataset(x, t)

# データセット分割
# 60%:学習用, 20%:検証用, 20%:テスト用
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# ミニバッチに分割
batch_size = 10
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

# 4→4→3のニューラルネットワーク
class Net(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)
    
    # 順伝播
    def forward(self, x):
        # 第1層
        h = self.fc1(x) # 線形変換
        h = F.relu(h) # 活性化関数
        # 第2層
        h = self.fc2(h) # 線形変換
        return h

# エポック数（学習回数）
max_epoch = 1

# モデルのインスタンス化とデバイスへの転送
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)

# 最適化手法
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

for epoch in range(max_epoch):
    for batch in train_loader:
        # ミニバッチの取得
        x, t = batch
        x, t = x.to(device), t.to(device)
        
        # 順伝播
        y = net(x)
        
        # 誤差の計算
        loss = F.cross_entropy(y, t)
        # print('loss:', loss)
        
        # 正解率の計算
        y_label = torch.argmax(y, dim=1)
        accracy = torch.mean((y_label == t).float())
        # print('accuracy:', accracy)
        
        # 勾配の初期化
        optimizer.zero_grad()
        
        # 逆伝播
        loss.backward()
        
        # パラメータの更新
        optimizer.step()

# 検証
net.eval()
with torch.no_grad():
    total_loss = 0
    total_accracy = 0
    for batch in val_loader:
        x, t = batch
        x, t = x.to(device), t.to(device)
        y = net(x)
        loss = F.cross_entropy(y, t)
        total_loss += loss.item()
        y_label = torch.argmax(y, dim=1)
        total_accracy += torch.mean((y_label == t).float())
    n = len(val_loader)
    print('val_loss:', total_loss / n)
    print('val_accuracy:', total_accracy / n)

val_loss: 0.6773030956586202
val_accuracy: tensor(0.7667)


In [78]:
import pytorch_lightning as pl
from torchmetrics.functional import accuracy
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris

# 乱数のシードを設定
# torch.manual_seed(0)

# irisデータセットの読み込み
iris = load_iris()

# 入力値と目標値を抽出
x = iris.data
t = iris.target

# Tensorに変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

# 入力値と目標値をまとめる
# （[[[入力値1, 入力値2, ...], 目標値1], [[入力値1, 入力値2, ...], 目標値2], ...]）
dataset = torch.utils.data.TensorDataset(x, t)

# データセット分割
# 60%:学習用, 20%:検証用, 20%:テスト用
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

# ミニバッチに分割
batch_size = 10
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

# 4→4→3のニューラルネットワーク
class Net(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)
    
    # 順伝播
    def forward(self, x):
        # 第1層
        h = self.fc1(x) # 線形変換
        h = F.relu(h) # 活性化関数
        # 第2層
        h = self.fc2(h) # 線形変換
        return h
    
    # 学習データの損失関数
    def training_step(self, batch, batch_idx):
        loss, acc = self.step(batch)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    # 検証データの損失関数
    def validation_step(self, batch, batch_idx):
        loss, acc = self.step(batch)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    # テストデータの損失関数
    def test_step(self, batch, batch_idx):
        loss, acc = self.step(batch)
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def step(self, batch):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = accuracy(y_label, t, 'multiclass', num_classes=3)
        return loss, acc
    
    # 最適化手法
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.01)

# pl.seed_everything(0)

# 学習の実行
net = Net()
trainer = pl.Trainer(max_epochs=30, deterministic=True)
trainer.fit(net, train_loader, val_loader)
trainer.callback_metrics

# テスト
trainer.test(dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | fc1  | Linear | 20    
1 | fc2  | Linear | 15    
--------------------------------
35        Trainable params
0         Non-trainable params
35        Total params
0.000     Total estimated model params size (MB)


Epoch 29: 100%|██████████| 9/9 [00:00<00:00, 96.39it/s, v_num=26, train_loss_step=0.379, train_acc_step=1.000, val_loss_step=0.409, val_acc_step=0.900, val_loss_epoch=0.465, val_acc_epoch=0.933, train_loss_epoch=0.397, train_acc_epoch=0.967] 

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 9/9 [00:00<00:00, 79.43it/s, v_num=26, train_loss_step=0.379, train_acc_step=1.000, val_loss_step=0.409, val_acc_step=0.900, val_loss_epoch=0.465, val_acc_epoch=0.933, train_loss_epoch=0.397, train_acc_epoch=0.967]


Restoring states from the checkpoint path at /workspaces/python/src/lightning_logs/version_26/checkpoints/epoch=29-step=270.ckpt
Loaded model weights from the checkpoint at /workspaces/python/src/lightning_logs/version_26/checkpoints/epoch=29-step=270.ckpt


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 43.93it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.9666666388511658
     test_loss_epoch        0.42295554280281067
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 0.42295554280281067,
  'test_acc_epoch': 0.9666666388511658}]